<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/2_03_Taller_Memoria_Gemini25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller: Memoria en LangChain 1.0 usando gemini-2.5-flash-lite
Este cuaderno explica cómo funciona la memoria en LangChain 1.0 mediante `RunnableWithMessageHistory`, `ChatMessageHistory` y Gemini.

## Configura tu clave de Gemini

In [ ]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")



## Instalación

In [ ]:
!pip install -q -U langchain langchain_community langchain_core langchain-google-genai

print("✅ Paquetes instalados correctamente")

In [ ]:
pip show langchain

## Cargar modelo gemini-2.5-flash-lite

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import time

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.3,
    max_retries=2
)

print("✅ Modelo configurado: gemini-2.5-flash-lite")


 ## Función de Manejo de Errores

In [ ]:
def invocar_con_retry(chain, input_data, config, max_intentos=3, delay=10):
    """Maneja automáticamente los errores 429 con esperas progresivas."""
    for intento in range(max_intentos):
        try:
            return chain.invoke(input_data, config=config)
        except Exception as e:
            error_msg = str(e)
            if "429" in error_msg or "ResourceExhausted" in error_msg:
                if intento < max_intentos - 1:
                    print(f"⚠️ Límite alcanzado. Esperando {delay} segundos...")
                    time.sleep(delay)
                    delay *= 2
                else:
                    print("❌ Se agotaron los intentos. Espera unos minutos.")
                    raise
            else:
                raise

print("✅ Función de retry configurada")

## ¿Qué es la memoria en LangChain 1.0?
LangChain 1.0 *ya no usa* `ConversationBufferMemory` o `ConversationSummaryMemory`.

En su lugar usa:

### ✔ `RunnableWithMessageHistory`
Un envoltorio que permite agregar historial a *cualquier* cadena, prompt o modelo.

### ✔ `ChatMessageHistory`
Objeto que guarda mensajes pasados entre usuario y asistente.

### ✔ El historial es **externo**, no está dentro del modelo.
Se simula un estado de conversación agregando mensajes previos en cada invocación.

## Configurar Memoria con RunnableWithMessageHistory

In [ ]:
# ⭐ PROMPT MÁS ESTRICTO - Evita alucinaciones
prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente honesto y preciso.

REGLAS ESTRICTAS QUE DEBES SEGUIR:
1. SOLO puedes acceder a la información que está en los mensajes del historial de esta conversación
2. Si el historial está vacío o no contiene la información solicitada, debes decir "No tengo esa información en el historial de esta conversación"
3. NUNCA inventes, asumas o adivines información
4. NUNCA digas que recuerdas algo si no está explícitamente en el historial
5. Si te preguntan sobre información que no está en los mensajes previos, responde: "No encuentro esa información en nuestro historial de conversación. ¿Podrías compartirla conmigo?"

Recuerda: Si el historial no contiene la respuesta, debes ser honesto y decir que no la tienes."""),
    MessagesPlaceholder(variable_name="messages"),
    ("user", "{input}")
])

stores = {}

def obtener_historial(session_id):
    if session_id not in stores:
        stores[session_id] = ChatMessageHistory()
    return stores[session_id]

chain_con_memoria = RunnableWithMessageHistory(
    prompt | model,
    obtener_historial,
    input_messages_key="input",
    history_messages_key="messages"
)

print("✅ Memoria configurada con prompt ANTI-ALUCINACIÓN")

In [ ]:
# PROMPT DETALLADO - Más explícito sobre recordar información
prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente amable y servicial con excelente memoria.

INSTRUCCIONES IMPORTANTES:
- Debes recordar TODA la información que el usuario te comparta
- Cuando el usuario te pregunte sobre información previa, busca en el historial de la conversación
- Si el usuario pregunta "¿En qué trabajo?" o similar, responde basándote en lo que ÉL te dijo anteriormente
- NUNCA inventes información que el usuario no te haya dicho
- SIEMPRE usa la información del historial de esta conversación específica

Recuerda: Cada conversación es independiente y debes recordar lo que el usuario de ESTA sesión te ha dicho."""),
    MessagesPlaceholder(variable_name="messages"), # ⬅️ ESTO ES CLAVE
    ("user", "{input}")
])

stores = {}

def obtener_historial(session_id):
    """Obtiene o crea un historial para una sesión específica."""
    if session_id not in stores:
        stores[session_id] = ChatMessageHistory()
    return stores[session_id]

# Crear cadena con memoria
chain_con_memoria = RunnableWithMessageHistory(
    prompt | model,
    obtener_historial,
    input_messages_key="input",
    history_messages_key="messages" # ⬅️ Debe coincidir con MessagesPlaceholder
)

print("✅ Memoria configurada con prompt DETALLADO y con ANTI-ALUCINACIÓN")

## Ejemplo 1 - Memoria Básica

In [ ]:
print("="*50)
print("EJEMPLO 1: MEMORIA BÁSICA")
print("="*50)

session_id = "usuario1"

try:
    print("\n🤖 Primera interacción:")
    respuesta1 = invocar_con_retry(
        chain_con_memoria,
        {"input": "Hola, mi nombre es Carlos y estudio Deep Learning."},
        {"configurable": {"session_id": session_id}}
    )
    print("Asistente:", respuesta1.content)

    print("\n⏳ Esperando 5 segundos...")
    time.sleep(5)

    print("\n🤖 Segunda interacción (prueba de memoria):")
    respuesta2 = invocar_con_retry(
        chain_con_memoria,
        {"input": "¿Recuerdas mi nombre y qué estudio?"},
        {"configurable": {"session_id": session_id}}
    )
    print("Asistente:", respuesta2.content)

except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 Espera 1-2 minutos antes de volver a intentar")

## Ejemplo 2: Memoria por múltiples usuarios

In [ ]:
print("\n" + "="*50)
print("EJEMPLO 2: MÚLTIPLES USUARIOS")
print("="*50)

try:
    # Usuario A
    print("\n👤 Sesión A - Ana:")
    time.sleep(5)

    print("Ana dice: Soy Ana y mi color favorito es el azul.")
    respA1 = invocar_con_retry(
        chain_con_memoria,
        {"input": "Soy Ana y mi color favorito es el azul."},
        {"configurable": {"session_id": "A"}}
    )
    print("🤖 Asistente (A):", respA1.content)

    time.sleep(5)
    print("\nAna pregunta: ¿Cuál es mi color favorito?")
    respA2 = invocar_con_retry(
        chain_con_memoria,
        {"input": "¿Cuál es mi color favorito?"},
        {"configurable": {"session_id": "A"}}
    )
    print("🤖 Asistente (A):", respA2.content)

    # Usuario B
    print("\n👤 Sesión B - Pedro:")
    time.sleep(5)

    print("Pedro dice: Me llamo Pedro y trabajo en Inteligencia Artificial.")
    respB1 = invocar_con_retry(
        chain_con_memoria,
        {"input": "Me llamo Pedro y trabajo en Inteligencia Artificial."},
        {"configurable": {"session_id": "B"}}
    )
    print("🤖 Asistente (B):", respB1.content)

    time.sleep(5)
    print("\nPedro pregunta: ¿En qué trabajo yo?")  # ⬅️ Más explícito
    respB2 = invocar_con_retry(
        chain_con_memoria,
        {"input": "¿En qué trabajo yo?"},  # ⬅️ Cambiado para ser más claro
        {"configurable": {"session_id": "B"}}
    )
    print("🤖 Asistente (B):", respB2.content)

    # Verificación adicional
    print("\n" + "-"*50)
    print("VERIFICACIÓN DE MEMORIA:")
    print("-"*50)

    time.sleep(5)
    print("\nAna pregunta: ¿Recuerdas mi nombre?")
    respA3 = invocar_con_retry(
        chain_con_memoria,
        {"input": "¿Recuerdas mi nombre?"},
        {"configurable": {"session_id": "A"}}
    )
    print("🤖 Asistente (A):", respA3.content)

    time.sleep(5)
    print("\nPedro pregunta: ¿Cuál es mi nombre y profesión?")
    respB3 = invocar_con_retry(
        chain_con_memoria,
        {"input": "¿Cuál es mi nombre y profesión?"},
        {"configurable": {"session_id": "B"}}
    )
    print("🤖 Asistente (B):", respB3.content)

    print("\n✅ Ejemplo completado - Verifica que cada sesión mantenga su memoria independiente")

except Exception as e:
    print(f"❌ Error: {e}")